In [1]:
import tensorflow as tf

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import tensorflow_hub as hub

In [1]:
from glob import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from collections import defaultdict
from scipy import spatial

In [2]:
data_path = "/home/shruti/Desktop/iitgn/courses/SEM2/ML/Project/code/PaperAcceptancePrediction/ICLR data/masterdata_unbalanced/"

years = [2017, 2018, 2019, 2020]
rev_dict = {}
paper_dict = {}
dec_dict = {}
iclr_arxiv_map = {}

for y in years:
    rev_dict[y] = pd.read_pickle(data_path + "off_rev_dict_{}.pkl".format(y))
    paper_dict[y] = pd.read_pickle(data_path + "papers_{}.pkl".format(y))
    dec_dict[y] = pd.read_pickle(data_path + "paper_decision_dict_{}.pkl".format(y))

iclr_arxiv_map = pd.read_pickle("./data/iclr_arxiv_map.pkl")

# Dataset

We were thinking about this problem for EACL:
    
Given a review text, can we identify the text span that talks about the comparisons.
After identification, can we identify different aspects associated with it:
1. Is it positive or negative?
2. What module does it talks about (dataset, model, metric, etc.)
3. Does it suggests some papers to cite?
4. We can also suggest the overlap between different reviews in terms of comparison comments. 
This can help us understanding how much reviews are coherent

Motivation: The system would be more useful for metareviewers. Authors can also be benefited, but they 
can always read the review and get this info.  We can always say that with the increase in exponential 
growth in conference submissions, the evaluation task for meta-reviewers is becoming more and more hard. 
The current system would help the metareviews task in quickly glancing the multiple reviews for a given paper.

## Create a test set of reviews for manual annotation

### Phase 2

In [5]:
test_set = ["2020_r1e_FpNFDr", "2020_SyevYxHtDB", "2019_SyxZJn05YX", "2019_BJx0sjC5FX", "2018_SkZxCk-0Z", 
            "2018_HkfXMz-Ab", "2017_B1ckMDqlg", "2017_HJ0NvFzxl", "2020_B1lsXREYvr", "2020_rkltE0VKwH", 
            "2018_Hki-ZlbA-", "2018_rJBiunlAW", "2019_rJzoujRct7", "2019_SJf_XhCqKm", "2017_Bk0MRI5lg",
            "2017_BJ9fZNqle"]


# list(iclr_arxiv_map.keys())[0:20]
counter = 0

for k, v in iclr_arxiv_map.items():
    if not k in test_set:
        if v["label"] == "Reject":
            print(k)
            counter += 1
        if counter > 2:
            break

2019_HylsgnCcFQ
2020_H1x-3xSKDr
2019_rJedbn0ctQ


In [36]:
test_set_augmentation_2ndphase = [
    # Rejected
    "2020_Byg79h4tvB", "2020_r1eX1yrKwB", "2019_HyVxPsC9tm", "2019_H1lFZnR5YX", "2018_SyYYPdg0-", 
    "2018_HyHmGyZCZ", "2017_BJAA4wKxg", "2017_r1y1aawlg",
    # Accepted
    "2020_BkeWw6VFwr", "2020_HkgsPhNYPS", "2019_HylTBhA5tQ", "2019_B1l08oAct7", "2018_H135uzZ0-", 
    "2018_HyUNwulC-", "2017_S1_pAu9xl", "2017_H1oyRlYgg"
]

train_set = [
    # Rejected
    "2020_SJegkkrYPS", "2020_ryen_CEFwr", "2019_ByxAOoR5K7", "2019_r1eO_oCqtQ", "2018_Bk6qQGWRb", 
    "2018_H1LAqMbRW", "2017_ByToKu9ll", "2017_H1Gq5Q9el",
    # Accepted
    "2020_ryxC6kSYPr", "2020_rJld3hEYvS", "2019_r1eEG20qKQ", "2019_HJfwJ2A5KX", "2018_BJ8c3f-0b", 
    "2018_SJx9GQb0-",  "2017_S1Bb3D5gg", "2017_HyTqHL5xg"
]

In [37]:
sents_list = defaultdict(dict)

for k in test_set_augmentation_2ndphase:
    if k in iclr_arxiv_map:
        y = int(k.split("_")[0])
        k_wo_y = k.split("_", 1)[1]
        
        sents_list[k]["pid"] = k
        sents_list[k]["sents"] = []
        sents_list[k]["dec"] = iclr_arxiv_map[k]["label"]
        
        for rev_num in rev_dict[y][k_wo_y]:
            rev_text = rev_num["content"]["review"]

            rev_text = rev_text.strip()
            
            
            rev_text = re.sub("(?<=[ \(])e[\.]?g[\.]?:? ", "eg: ", rev_text)
            rev_text = re.sub(" et[\.]? al[\.]", " et al", rev_text)
            rev_text = re.sub("(?<=[ \(])i[\.]?e[\.] ", "ie ", rev_text)
            rev_text = re.sub("(?<=[ \(])Fig[\.]? ", "Figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])fig[\.]? ", "figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Alg\. ", "Algorithm ", rev_text)
            rev_text = re.sub("Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])eq\. ", "equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Sec[\.]? ", "Section ", rev_text)
            rev_text = re.sub("(?<=[ \(])sec\. ", "section ", rev_text)
            rev_text = re.sub("(?<=: [0-9])(\.) ", ")", rev_text)
            rev_text = re.sub("(?<=; [0-9])(\.) ", ")", rev_text)
            rev_text = re.sub("(?<=\n[0-9])(\.) ", ")", rev_text)
            ####rev_text = re.sub("(?<=[0-9])(\.) ", ") ", rev_text)
            rev_text = re.sub("\n\\n", "\n", rev_text)

            sent_text = nltk.sent_tokenize(rev_text)
            for s in sent_text:
                sents_list[k]["sents"].append(s)
    else:
        print("Not found: %s, use another key"%k)

In [38]:
sents_list.keys(), len(list(sents_list.keys()))

(dict_keys(['2017_S1_pAu9xl', '2018_SyYYPdg0-', '2017_BJAA4wKxg', '2019_HyVxPsC9tm', '2019_HylTBhA5tQ', '2019_B1l08oAct7', '2018_H135uzZ0-', '2017_H1oyRlYgg', '2017_r1y1aawlg', '2020_r1eX1yrKwB', '2020_Byg79h4tvB', '2019_H1lFZnR5YX', '2020_BkeWw6VFwr', '2018_HyHmGyZCZ', '2018_HyUNwulC-', '2020_HkgsPhNYPS']),
 16)

In [39]:
df = pd.DataFrame(columns=['pid', 'dec', 'sent'])

i = 0
for k in sents_list:
    dec = sents_list[k]["dec"]
    pid = sents_list[k]["pid"]
    for s in sents_list[k]["sents"]:
        df.loc[i] = [pid, dec, s]
        i += 1

df.to_excel("testset_16_phase2Ann.xlsx")
df.head()

,pid,dec,sent
0,2017_S1_pAu9xl,Accept,The paper shows a different approach to a tern...
1,2017_S1_pAu9xl,Accept,Strengths:\n1)The paper shows performance impr...
2,2017_S1_pAu9xl,Accept,Weaknesses:\n1)The paper is very incremental.
3,2017_S1_pAu9xl,Accept,2)The paper is addressed to a very narrow audi...
4,2017_S1_pAu9xl,Accept,The paper very clearly assumes that the reader...


In [40]:
sents_list = defaultdict(dict)

for k in train_set:
    if k in iclr_arxiv_map:
        y = int(k.split("_")[0])
        k_wo_y = k.split("_", 1)[1]
        
        sents_list[k]["pid"] = k
        sents_list[k]["sents"] = []
        sents_list[k]["dec"] = iclr_arxiv_map[k]["label"]
        
        for rev_num in rev_dict[y][k_wo_y]:
            rev_text = rev_num["content"]["review"]

            rev_text = rev_text.strip()
            
            
            rev_text = re.sub("(?<=[ \(])e[\.]?g[\.]?:? ", "eg: ", rev_text)
            rev_text = re.sub(" et[\.]? al[\.]", " et al", rev_text)
            rev_text = re.sub("(?<=[ \(])i[\.]?e[\.] ", "ie ", rev_text)
            rev_text = re.sub("(?<=[ \(])Fig[\.]? ", "Figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])fig[\.]? ", "figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Alg\. ", "Algorithm ", rev_text)
            rev_text = re.sub("Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])eq\. ", "equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Sec[\.]? ", "Section ", rev_text)
            rev_text = re.sub("(?<=[ \(])sec\. ", "section ", rev_text)
            rev_text = re.sub("(?<=: [0-9])(\.) ", ")", rev_text)
            rev_text = re.sub("(?<=; [0-9])(\.) ", ")", rev_text)
            rev_text = re.sub("(?<=\n[0-9])(\.) ", ")", rev_text)
            ####rev_text = re.sub("(?<=[0-9])(\.) ", ") ", rev_text)
            rev_text = re.sub("\n\\n", "\n", rev_text)

            sent_text = nltk.sent_tokenize(rev_text)
            for s in sent_text:
                sents_list[k]["sents"].append(s)
    else:
        print("Not found: %s, use another key"%k)

In [41]:
sents_list.keys(), len(list(sents_list.keys()))

(dict_keys(['2019_r1eO_oCqtQ', '2019_HJfwJ2A5KX', '2017_ByToKu9ll', '2017_HyTqHL5xg', '2018_H1LAqMbRW', '2020_ryen_CEFwr', '2020_ryxC6kSYPr', '2017_H1Gq5Q9el', '2020_SJegkkrYPS', '2017_S1Bb3D5gg', '2020_rJld3hEYvS', '2018_SJx9GQb0-', '2018_Bk6qQGWRb', '2019_r1eEG20qKQ', '2018_BJ8c3f-0b', '2019_ByxAOoR5K7']),
 16)

In [42]:
df = pd.DataFrame(columns=['pid', 'dec', 'sent'])

i = 0
for k in sents_list:
    dec = sents_list[k]["dec"]
    pid = sents_list[k]["pid"]
    for s in sents_list[k]["sents"]:
        df.loc[i] = [pid, dec, s]
        i += 1

df.to_excel("trainset_16_phase2Ann.xlsx")
df.head()

,pid,dec,sent
0,2019_r1eO_oCqtQ,Reject,In the vein of recent work on learning “tickin...
1,2019_r1eO_oCqtQ,Reject,The performance of the modified g-LSTM is comp...
2,2019_r1eO_oCqtQ,Reject,The authors argue that g-LSTM results in bette...
3,2019_r1eO_oCqtQ,Reject,"Additionally, it is proposed that one can redu..."
4,2019_r1eO_oCqtQ,Reject,"Finally, a technique for gradually transitioni..."


### Phase 1

In [82]:
test_set = ["2020_r1e_FpNFDr", "2020_SyevYxHtDB", "2019_SyxZJn05YX", "2019_BJx0sjC5FX", "2018_SkZxCk-0Z", 
            "2018_HkfXMz-Ab", "2017_B1ckMDqlg", "2017_HJ0NvFzxl", "2020_B1lsXREYvr", "2020_rkltE0VKwH", 
            "2018_Hki-ZlbA-", "2018_rJBiunlAW", "2019_rJzoujRct7", "2019_SJf_XhCqKm", "2017_Bk0MRI5lg",
            "2017_BJ9fZNqle"]

sents_list = defaultdict(dict)

for k in test_set:
    if k in iclr_arxiv_map:
        y = int(k.split("_")[0])
        k_wo_y = k.split("_", 1)[1]
        
        sents_list[k]["pid"] = k
        sents_list[k]["sents"] = []
        sents_list[k]["dec"] = iclr_arxiv_map[k]["label"]
        
        for rev_num in rev_dict[y][k_wo_y]:
            rev_text = rev_num["content"]["review"]

            rev_text = rev_text.strip()
            
            
            rev_text = re.sub("(?<=[ \(])e[\.]?g[\.]?:? ", "eg: ", rev_text)
            rev_text = re.sub(" et[\.]? al[\.]", " et al", rev_text)
            rev_text = re.sub("(?<=[ \(])i[\.]?e[\.] ", "ie ", rev_text)
            rev_text = re.sub("(?<=[ \(])Fig[\.]? ", "Figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])fig[\.]? ", "figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Alg\. ", "Algorithm ", rev_text)
            rev_text = re.sub("Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])eq\. ", "equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Sec[\.]? ", "Section ", rev_text)
            rev_text = re.sub("(?<=[ \(])sec\. ", "section ", rev_text)
            rev_text = re.sub("(?<=: [0-9])(\.) ", ")", rev_text)
            rev_text = re.sub("(?<=; [0-9])(\.) ", ")", rev_text)
            ####rev_text = re.sub("(?<=[0-9])(\.) ", ") ", rev_text)
            rev_text = re.sub("\n\\n", "\n", rev_text)

            sent_text = nltk.sent_tokenize(rev_text)
            for s in sent_text:
                sents_list[k]["sents"].append(s)
    else:
        print("Not found: %s, use another key"%k)

In [83]:
sents_list.keys(), len(list(sents_list.keys()))

(dict_keys(['2019_SJf_XhCqKm', '2017_Bk0MRI5lg', '2020_SyevYxHtDB', '2018_rJBiunlAW', '2020_rkltE0VKwH', '2018_Hki-ZlbA-', '2019_BJx0sjC5FX', '2020_r1e_FpNFDr', '2020_B1lsXREYvr', '2018_SkZxCk-0Z', '2019_rJzoujRct7', '2018_HkfXMz-Ab', '2017_BJ9fZNqle', '2019_SyxZJn05YX', '2017_B1ckMDqlg', '2017_HJ0NvFzxl']),
 16)

In [84]:
df = pd.DataFrame(columns=['pid', 'dec', 'sent'])

### Save the test set of 16 reviews for manual annotation

In [85]:
i = 0
for k in sents_list:
    dec = sents_list[k]["dec"]
    pid = sents_list[k]["pid"]
    for s in sents_list[k]["sents"]:
        df.loc[i] = [pid, dec, s]
        i += 1

In [86]:
df.head(5)

,pid,dec,sent
0,2019_SJf_XhCqKm,Reject,The authors propose to use k-DPP to select a s...
1,2019_SJf_XhCqKm,Reject,"This paper covers the related work nicely, wit..."
2,2019_SJf_XhCqKm,Reject,The rest of the paper are also clearly written.
3,2019_SJf_XhCqKm,Reject,"However, I have some concerns about the propos..."
4,2019_SJf_XhCqKm,Reject,"- It is not clear how to define the kernel, th..."


In [87]:
df.to_excel("testset_16.xlsx")

# ============================================================

### Read the initial set of sentences

In [9]:
df = pd.read_excel("data/initialsetmcomp.xlsx")

In [10]:
df.size, df.head()

(117,          pid                                               sent  mcomp
 0  S1HcOI5le  The idea of the paper is interesting there are...      1
 1  S1HcOI5le  It's not clear how this method compares agains...      1
 2  S1HcOI5le  Measure: Accuracy difference does not look lik...      1
 3  S1HcOI5le  Instead the authors could position this work a...      1
 4  S1HcOI5le  If the authors care to compare their approach ...      1)

### Use above to find sentences from 2020 reviews for 10 papers

In [19]:
sents_dict = defaultdict(list)
stop = 1

for y in [2020]:
    for k in rev_dict[y]:
        
        if stop > 100:
            break
        stop += 1 
            
        year_key = str(y) + "_" + k
        if year_key in iclr_arxiv_map:
            for rev_num in rev_dict[y][k]:
                rev_text = rev_num["content"]["review"]
                
                rev_text = re.sub(" e[\.]?g[\.]?:? ", " eg: ", rev_text)
                rev_text = re.sub(" et[\.]? al[\.]", " et al", rev_text)
                rev_text = re.sub(" i[\.]?e[\.] ", " ie ", rev_text)
                rev_text = re.sub("\\n", " ", rev_text)
                
                sent_text = nltk.sent_tokenize(rev_text)
                
                for s in sent_text:
                    sents_dict[year_key].append(s)

In [20]:
with open("data/2020_test_sents.pkl", "wb") as f:
    pickle.dump(sents_dict, f)

## Read directly

In [11]:
with open("data/2020_test_sents.pkl", "rb") as f:
    sents_dict = pickle.load(f)

## SSim with USE

In [13]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [16]:
initial_mcomp_set = list(df["sent"])

In [17]:
initial_embs = embed(initial_mcomp_set)

In [23]:
print(initial_embs[0].shape)

(512,)


In [25]:
rev_embeddings = defaultdict(list)

for k, v in sents_dict.items():
    rev_embeddings[k] = embed(v)

In [26]:
rev_embeddings.keys()

dict_keys(['2020_Syx-bCEFPS', '2020_Bkf4XgrKvS', '2020_Sygn20VtwH', '2020_ryl0cAVtPH', '2020_SJe3KCNKPr', '2020_Bylx-TNKvH', '2020_r1lEd64YwH', '2020_rJx1Na4Fwr', '2020_SklOUpEYvB', '2020_H1ebhnEYDH', '2020_rJx7wlSYvB', '2020_rJxAo2VYwr', '2020_HJe_yR4Fwr', '2020_ryl3ygHYDB', '2020_SJxWS64FwH', '2020_HygDF1rYDB', '2020_H1ekF2EYDH', '2020_r1lnigSFDr', '2020_ryx4TlHKDS', '2020_S1xJikHtDH', '2020_Bkl8YR4YDB', '2020_B1eP504YDr', '2020_SJeLIgBKPS', '2020_Hyg9anEFPS', '2020_HJeTo2VFwH', '2020_BklhsgSFvB', '2020_BylaUTNtPS', '2020_rylfl6VFDH', '2020_rkg8xTEtvB', '2020_BkgUB1SYPS', '2020_BkgZxpVFvH', '2020_SJeC2TNYwB', '2020_Hke0oa4KwS', '2020_HJlWWJSFDH', '2020_rJg3zxBYwH', '2020_B1elqkrKPH', '2020_rylvAA4YDB', '2020_SJgob6NKvH', '2020_Hye4WaVYwr', '2020_SJgs8TVtvr', '2020_r1l1myStwr', '2020_rklEj2EFvB', '2020_HkgsPhNYPS', '2020_B1gXYR4YDH', '2020_BJlkgaNKvr', '2020_B1l0wp4tvr', '2020_HJedXaEtvS', '2020_HJxrVA4FDS', '2020_rJx8I1rFwr', '2020_r1ltgp4FwS', '2020_SJlVY04FwH', '2020_r1lczkHKPr', '

In [30]:
for s in sents_dict["2020_Hke0oa4KwS"]:
    if s.find("compar") > -1:
        print(s)

But the most challenging part of the research in this direction is that, although the theory suggests we can get nice confidence information (in population), when we use the deep neural networks, the quality of confidence estimation can be very bad (overconfidence in empirical estimation) compared with the high accuracy we can achieve.
 The paper provides a comparison of several uncertainty measures that have been proposed for neural networks.
The authors propose the use of expected Bayes factors as aggregate measures of how much information is conveyed by a measure of uncertainty, and compares several proposed approaches based on this measure on CIFAR-10/100 and ImageNet-1k.
The use of Bayes factors for measuring the quality of the uncertainty estimates, and the comparison of various existing methods on that measure seems useful.
Overall, even if there is no clear evidence that one method to evaluate uncertainty is consistently better than the others, I feel that the idea of using Bay

In [29]:
sents_dict["2020_Hke0oa4KwS"], len(rev_embeddings["2020_Hke0oa4KwS"])

(['Summary: This paper proposes an uncertainty measure called an implied loss.',
  'The authors suggest that it is a simple way to quantify the uncertainty of the model.',
  'It is suggested that "Low implied loss (uncertainty) means a high probability of correct classification on the test set.".',
  'They suggest that the analysis of an implied loss justifies the maximum confidence value of softmax-cross entropy.',
  'They also extend to evaluate Top-k uncertainty (the uncertainty whether our prediction is in the Top-5 maximum values of our confidence score or not).',
  '======================================================== Clarity: I found that this paper content does not seem to be difficult mathematically, however, it is difficult to follow the paper and here I list several parts that can potentially be improved:  1.',
  'INTRO: the sentence "Our implied loss interpretation justifies both methods, since we demonstrate that "both these quantities" are uncertainty measures.".',
  

In [53]:
sims = defaultdict(dict)

for k in rev_embeddings:
    for _, i in enumerate(rev_embeddings[k]):
        sims[k][_] = []
        for sent_emb in initial_embs:
            sims[k][_].append(np.inner(i, sent_emb))

In [54]:
max_sims_per_sent = defaultdict(list)
for k in sims:
    local_sent_sims = []
    for sid in sims[k]:
        local_sent_sims.append((sid, max(sims[k][sid])))
    local_sent_sims_sorted = sorted(local_sent_sims, key=lambda x: x[1], reverse=True)
    max_sims_per_sent[k] = local_sent_sims_sorted

In [57]:
all_maxs = []
for k in max_sims_per_sent:
    all_maxs.append(max_sims_per_sent[k][0][1])
print(max(all_maxs), min(all_maxs))
print(sorted(all_maxs, reverse=True))

0.6950005 0.44239438
[0.6950005, 0.6543123, 0.6250528, 0.61709094, 0.6132989, 0.60757875, 0.5873779, 0.57469136, 0.5672308, 0.5643765, 0.56402373, 0.5636523, 0.5614678, 0.56079316, 0.55891705, 0.55869263, 0.55829895, 0.55511045, 0.5549477, 0.5543808, 0.5518889, 0.55046445, 0.5497707, 0.54956484, 0.5495119, 0.5463648, 0.5454315, 0.5449186, 0.5416362, 0.53818166, 0.5375632, 0.5361639, 0.5358979, 0.5353979, 0.5346217, 0.5345441, 0.5338491, 0.53194886, 0.5292664, 0.5270127, 0.5243573, 0.5241016, 0.5239669, 0.5222249, 0.52117246, 0.5201142, 0.51940846, 0.5186783, 0.51826525, 0.51785743, 0.5137969, 0.5111599, 0.50997114, 0.5092687, 0.508452, 0.5078472, 0.5073638, 0.5056021, 0.505427, 0.5048554, 0.5040755, 0.5015641, 0.5010345, 0.50088227, 0.50031805, 0.49547097, 0.49505472, 0.49421135, 0.49331182, 0.49257076, 0.4917462, 0.48961833, 0.48952138, 0.48946032, 0.48726648, 0.48720586, 0.48191735, 0.47699964, 0.47338888, 0.47296995, 0.47137085, 0.46578664, 0.4632785, 0.4620925, 0.4559152, 0.4526633

In [153]:
max_sims_per_sent["2020_Bkf4XgrKvS"][0:10]

[(31, 0.47699964),
 (43, 0.47099784),
 (13, 0.44567394),
 (48, 0.43486825),
 (51, 0.42300326),
 (0, 0.41074166),
 (46, 0.41057122),
 (11, 0.40413868),
 (32, 0.40090787),
 (28, 0.3968842)]

In [81]:
eval_df = pd.DataFrame(columns=["pid", "sent", "sim"])

In [84]:
for i in max_sims_per_sent:
    for sent_num in max_sims_per_sent[i][0:5]:
        eval_df = eval_df.append({"pid": i, "sent": sents_dict[i][sent_num[0]], "sim": sent_num[1]}, ignore_index=True)

In [85]:
eval_df.head()

,pid,sent,sim
0,2020_Syx-bCEFPS,"2, we observe that the test accuracy for fine-...",0.507847
1,2020_Syx-bCEFPS,=========== Summary: This paper introduces two...,0.453779
2,2020_Syx-bCEFPS,"In this paper, the authorsestablish a large be...",0.453430
3,2020_Syx-bCEFPS,This paper contributes a new dataset for testi...,0.442424
4,2020_Syx-bCEFPS,"> Inception-ResNet-V2 (Szegedy et al, 2017) is...",0.425780


In [88]:
eval_df.to_excel("evaluate_use_sims.xlsx")

In [89]:
pwd

'/home/singh_shruti/workspace/meaningful_comparison/meaningful-comparison/mcomp_text_extraction'

In [141]:
eval_df_top10 = pd.DataFrame(columns=["pid", "sent", "sim"])

In [142]:
for i in max_sims_per_sent:
    for sent_num in max_sims_per_sent[i][0:10]:
        eval_df_top10 = eval_df_top10.append({"pid": i, "sent": sents_dict[i][sent_num[0]], "sim": sent_num[1]}, ignore_index=True)

In [143]:
eval_df_top10.to_excel("evaluate_use_sims_top10_sents.xlsx")

In [140]:
max_sims_per_sent["2020_Syx-bCEFPS"][0:20]

[(40, 0.5078472),
 (0, 0.45377856),
 (25, 0.45342988),
 (30, 0.44242394),
 (41, 0.42578),
 (26, 0.4224671),
 (28, 0.41788375),
 (1, 0.41554937),
 (14, 0.4098686),
 (23, 0.409846),
 (21, 0.40348357),
 (18, 0.39842135),
 (31, 0.39458615),
 (11, 0.38681322),
 (8, 0.3792917),
 (7, 0.3791846),
 (16, 0.37650996),
 (35, 0.3764558),
 (39, 0.3650995),
 (20, 0.3583181)]

# Extract top 10 sentences for each review using keyword based filtering

In [150]:
keywords = ["baseline", "compar", "et al", "sota", "state of the art", "state-of-the-art", "underperform", "outperform"]

In [156]:
kw_sims = defaultdict(dict)

for k in rev_embeddings:
    for _, i in enumerate(rev_embeddings[k]):
        for kw in keywords:
            if sents_dict[k][_].lower().find(kw) > -1:
                kw_sims[k][_] = sims[k][_]
                break

In [157]:
len(rev_embeddings["2020_Bkf4XgrKvS"]), len(sents_dict["2020_Bkf4XgrKvS"])

(56, 56)

In [158]:
max_sims_per_sent_with_kws = defaultdict(list)

for k in kw_sims:
    local_sent_sims = []
    for sid in kw_sims[k]:
        local_sent_sims.append((sid, max(kw_sims[k][sid])))
    local_sent_sims_sorted = sorted(local_sent_sims, key=lambda x: x[1], reverse=True)
    max_sims_per_sent_with_kws[k] = local_sent_sims_sorted

In [159]:
all_maxs_with_kws = []
for k in max_sims_per_sent_with_kws:
    all_maxs_with_kws.append(max_sims_per_sent_with_kws[k][0][1])
print(max(all_maxs_with_kws), min(all_maxs_with_kws))
print(sorted(all_maxs_with_kws, reverse=True))

0.6950005 0.37948126
[0.6950005, 0.6543123, 0.61709094, 0.6132989, 0.60757875, 0.5873779, 0.57469136, 0.5672308, 0.56402373, 0.56079316, 0.55891705, 0.55869263, 0.55046445, 0.5497707, 0.5495119, 0.5463648, 0.5454315, 0.53931415, 0.53818166, 0.53693724, 0.5358979, 0.5345441, 0.5338491, 0.5277494, 0.5241016, 0.52117246, 0.5194628, 0.51940846, 0.5186783, 0.51785743, 0.5137969, 0.5111274, 0.51009715, 0.50997114, 0.50939703, 0.508452, 0.5073638, 0.5062192, 0.5048554, 0.5015641, 0.50144523, 0.4999418, 0.49964732, 0.49880302, 0.49560162, 0.49547097, 0.49421135, 0.49203223, 0.49075627, 0.49026197, 0.48952138, 0.48946032, 0.48882446, 0.48563325, 0.48484242, 0.48116815, 0.47912854, 0.47839898, 0.47706, 0.47338888, 0.47076106, 0.47059554, 0.4632785, 0.46246776, 0.4620925, 0.45951104, 0.4495302, 0.44873565, 0.44744566, 0.44567394, 0.44481036, 0.44366318, 0.4419094, 0.4401729, 0.43419603, 0.43233782, 0.42578, 0.4249261, 0.41760656, 0.40755904, 0.407412, 0.40453553, 0.39963016, 0.39556628, 0.3909938

In [166]:
eval_df_top10_with_kw = pd.DataFrame(columns=["pid", "sent", "sim"])

In [167]:
for i in max_sims_per_sent_with_kws:
    if len(max_sims_per_sent_with_kws[i]) > 9:
        for sent_num in max_sims_per_sent_with_kws[i][0:10]:
            eval_df_top10_with_kw = eval_df_top10_with_kw.append({"pid": i, "sent": sents_dict[i][sent_num[0]], "sim": sent_num[1]}, ignore_index=True)
    else:
        available_sents = []
        for sent_num in max_sims_per_sent_with_kws[i][0:10]:
            available_sents.append(sent_num[0])
            eval_df_top10_with_kw = eval_df_top10_with_kw.append({"pid": i, "sent": sents_dict[i][sent_num[0]], "sim": sent_num[1]}, ignore_index=True)
        
        total_count = len(available_sents)
        for sent_num in max_sims_per_sent[i]:
            if total_count < 9:
                if not sent_num[0] in available_sents:
                    eval_df_top10_with_kw = eval_df_top10_with_kw.append({"pid": i, "sent": sents_dict[i][sent_num[0]], "sim": sent_num[1]}, ignore_index=True)
                    total_count += 1
            else:
                break

In [168]:
eval_df_top10_with_kw.to_excel("evaluate_use_sims_top10_sents_with_kws_wo_duplicates.xlsx")

## Extract top 100 similar sentences

In [91]:
with open("2020_test_sents_ALL.pkl", "rb") as f:
    all_comp_potential_sents = pickle.load(f)

In [92]:
sum(len(all_comp_potential_sents[k]) for k in all_comp_potential_sents)

16879

In [98]:
sent_mappings = {}

for k in all_comp_potential_sents:
    for _, i in enumerate(all_comp_potential_sents[k]):
        sent_mappings[k + str(_)] = (i, embed([i]))
len(sent_mappings)

16879

In [99]:
list(sent_mappings.items())[0]

('2020_BkgM7xHYwH0',
 ('It would be helpful to include some experiments indicating that this was the case (for the baseline) and that this method does indeed help with this problem.',
  <tf.Tensor: id=3639130, shape=(1, 512), dtype=float32, numpy=
  array([[ 3.82600687e-02, -9.52417310e-03,  4.00788290e-03,
          -9.26006287e-02, -7.38939792e-02, -1.13683753e-02,
          -7.67421490e-03, -1.00507801e-02,  4.56398949e-02,
           6.65699244e-02, -3.76119651e-02,  1.87283847e-02,
          -6.73146977e-04,  5.91846816e-02, -4.64790314e-02,
           6.89068362e-02, -4.41112854e-02,  6.64667413e-02,
          -3.89787890e-02, -6.48101717e-02, -3.48372688e-03,
           6.52439073e-02, -3.50743085e-02,  4.53783758e-02,
          -5.39826229e-02,  2.79634837e-02,  2.07222048e-02,
          -8.30886513e-02,  3.16294953e-02,  2.58914847e-02,
           1.41347591e-02, -1.75061692e-02,  1.65479798e-02,
           5.91456145e-02, -2.45515034e-02,  7.63032734e-02,
          -6.0093194

In [122]:
# sims_dict = defaultdict(list)
# max_sims_per_sentence = defaultdict(int)
count = 0

for ite in sent_mappings:
    if not ite in max_sims_per_sentence:
        count += 1
        for sent_emb in initial_embs:
            sims_dict[ite].append(np.inner(sent_mappings[ite][1], sent_emb)[0])
        max_sims_per_sentence[ite] = max(sims_dict[ite])

        if count > 5000:
            break

In [123]:
sorted_max_sims = sorted(max_sims_per_sentence.items(), key=lambda x: x[1], reverse=True)

In [124]:
sorted_max_sims[0:5]

[('2020_BylRkAEKDH9', 0.7344125),
 ('2020_BJg4NgBKvH13', 0.7061932),
 ('2020_Hkg0olStDr1', 0.70500076),
 ('2020_Hkg0olStDr0', 0.7050007),
 ('2020_rylfl6VFDH4', 0.6950005)]

In [128]:
top_1000_df = pd.DataFrame(columns=['pid', 'sent', 'sim'])

In [129]:
for i in sorted_max_sims[0:1000]:
    top_1000_df = top_1000_df.append({"pid": i[0], "sent": sent_mappings[i[0]][0], "sim": i[1]}, ignore_index=True)

In [130]:
top_1000_df.to_excel("top_1000_similar_mcomp_sentences.xlsx")

In [135]:
s = "With this knowledge, we have run a large number of experiments (1,840) to faithfully compare the existing methods."
vec = embed([s])

In [137]:
for sent_emb in initial_embs:
    print(np.inner(vec, sent_emb)[0])

0.29652703
0.30603814
0.3059607
0.16510145
0.25347525
0.079709984
0.25372633
0.2122944
0.2943849
0.34907377
0.3447559
0.20344344
0.29056942
0.26877886
0.075643644
0.13323496
0.15270266
0.18165548
0.09214949
0.18762672
-0.04017473
0.19602987
0.25877717
0.26004434
0.26084474
-0.050904255
0.39915735
0.31180298
0.0858697
0.19444376
0.27616215
0.11376745
0.07652992
0.25433397
0.28549474
0.35478094
0.3464392
-0.0019737743
0.11035041


## Filtered 2020 test set (local copy)

In [169]:
keywords = ["baseline", "compar", "et al", "SOTA", "state of the art", "state-of-the-art", "underperform", "outperform"]

kw_poor = ["novel", "evaluat", "benchmark", "contribution", "contrast", "method", "result", "significan", 
           "approach", "performance", "technique", "report", "experiment", "propose", "model", "discuss", 
           "problem", "task", "metric", "score", "publication", "analyze", "analyse", "analysis", 
           "replicate", "submission"]

# &, et al, [1,2][7-9,0-1][0-9][0-9]

In [5]:
years

[2017, 2018, 2019, 2020]

In [6]:
sents_list = []
stop = False

for y in years[:-1]:
    if stop:
        break
    for k in rev_dict[y]:
        
        if len(sents_list) > 800*(y-2016) and y != 2019:
            change_year = True
            print("Changing year: ", y)
            break
        if len(sents_list) > 5000:
            stop = True
            break
            
        year_key = str(y) + "_" + k
        if year_key in iclr_arxiv_map:
            for rev_num in rev_dict[y][k]:
                rev_text = rev_num["content"]["review"]
                
                rev_text = re.sub(" e[\.]?g[\.]?:? ", " eg: ", rev_text)
                rev_text = re.sub(" et[\.]? al[\.]", " et al", rev_text)
                rev_text = re.sub(" i[\.]?e[\.] ", " ie ", rev_text)
                rev_text = re.sub("\\n", " ", rev_text)
                
                sent_text = nltk.sent_tokenize(rev_text)
                
                for s in sent_text:
                    for kw in keywords:
                        if s.find(kw) > -1:
                            sents_list.append((k, s))
                            break

Changing year:  2017
Changing year:  2018


In [7]:
df = pd.DataFrame(sents_list, columns=['pid', 'sentence'])

In [8]:
df.head(2)

,pid,sentence
0,BydrOIcle,"So, this comparison might just be showing that..."
1,SyOvg6jxx,Several points are appealing about this approa...


In [9]:
# df = df.sample(frac=1.0)

In [10]:
df.head(10)

,pid,sentence
0,BydrOIcle,"So, this comparison might just be showing that..."
1,SyOvg6jxx,Several points are appealing about this approa...
2,SyOvg6jxx,"In addition, there are results for comparison ..."
3,SyOvg6jxx,The results indicate that the approach clearly...
4,SyOvg6jxx,It seems like the technique could be easily us...
5,SyOvg6jxx,The paper addresses an important problem (expl...
6,SyOvg6jxx,It is a nice alternative approach to the one o...
7,SyOvg6jxx,"Specifically, I am not as concerned about beat..."
8,SyOvg6jxx,The figure S9 from Mnih et al points to instan...
9,SyOvg6jxx,"Without ""feature engineering"", the authors ach..."


In [29]:
df.to_excel("ann_comparison_only.xlsx")

## Create a test set of 20 reviews for manual annotation

In [17]:
# list(iclr_arxiv_map.keys())[0:20]
counter = 0

for k, v in iclr_arxiv_map.items():
    if v["label"] == "Reject":
        print(k)
        counter += 1
    if counter > 10:
        break

2020_B1lsXREYvr
2020_rkltE0VKwH
2018_Hki-ZlbA-
2018_rJBiunlAW
2018_B1nxTzbRZ
2019_rJzoujRct7
2019_SJf_XhCqKm
2018_BJ_QxP1AZ
2019_HklbTjRcKX
2020_Sygn20VtwH
2018_BkeC_J-R-


In [82]:
test_set = ["2020_r1e_FpNFDr", "2020_SyevYxHtDB", "2019_SyxZJn05YX", "2019_BJx0sjC5FX", "2018_SkZxCk-0Z", 
            "2018_HkfXMz-Ab", "2017_B1ckMDqlg", "2017_HJ0NvFzxl", "2020_B1lsXREYvr", "2020_rkltE0VKwH", 
            "2018_Hki-ZlbA-", "2018_rJBiunlAW", "2019_rJzoujRct7", "2019_SJf_XhCqKm", "2017_Bk0MRI5lg",
            "2017_BJ9fZNqle"]

sents_list = defaultdict(dict)

for k in test_set:
    if k in iclr_arxiv_map:
        y = int(k.split("_")[0])
        k_wo_y = k.split("_", 1)[1]
        
        sents_list[k]["pid"] = k
        sents_list[k]["sents"] = []
        sents_list[k]["dec"] = iclr_arxiv_map[k]["label"]
        
        for rev_num in rev_dict[y][k_wo_y]:
            rev_text = rev_num["content"]["review"]

            rev_text = rev_text.strip()
            
            
            rev_text = re.sub("(?<=[ \(])e[\.]?g[\.]?:? ", "eg: ", rev_text)
            rev_text = re.sub(" et[\.]? al[\.]", " et al", rev_text)
            rev_text = re.sub("(?<=[ \(])i[\.]?e[\.] ", "ie ", rev_text)
            rev_text = re.sub("(?<=[ \(])Fig[\.]? ", "Figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])fig[\.]? ", "figure ", rev_text)
            rev_text = re.sub("(?<=[ \(])Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Alg\. ", "Algorithm ", rev_text)
            rev_text = re.sub("Eq[\.]? ", "Equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])eq\. ", "equation ", rev_text)
            rev_text = re.sub("(?<=[ \(])Sec[\.]? ", "Section ", rev_text)
            rev_text = re.sub("(?<=[ \(])sec\. ", "section ", rev_text)
            rev_text = re.sub("(?<=: [0-9])(\.) ", ")", rev_text)
            rev_text = re.sub("(?<=; [0-9])(\.) ", ")", rev_text)
            ####rev_text = re.sub("(?<=[0-9])(\.) ", ") ", rev_text)
            rev_text = re.sub("\n\\n", "\n", rev_text)

            sent_text = nltk.sent_tokenize(rev_text)
            for s in sent_text:
                sents_list[k]["sents"].append(s)
    else:
        print("Not found: %s, use another key"%k)

In [83]:
sents_list.keys(), len(list(sents_list.keys()))

(dict_keys(['2019_SJf_XhCqKm', '2017_Bk0MRI5lg', '2020_SyevYxHtDB', '2018_rJBiunlAW', '2020_rkltE0VKwH', '2018_Hki-ZlbA-', '2019_BJx0sjC5FX', '2020_r1e_FpNFDr', '2020_B1lsXREYvr', '2018_SkZxCk-0Z', '2019_rJzoujRct7', '2018_HkfXMz-Ab', '2017_BJ9fZNqle', '2019_SyxZJn05YX', '2017_B1ckMDqlg', '2017_HJ0NvFzxl']),
 16)

In [84]:
df = pd.DataFrame(columns=['pid', 'dec', 'sent'])

### Save the test set of 20 reviews for manual annotation

In [85]:
i = 0
for k in sents_list:
    dec = sents_list[k]["dec"]
    pid = sents_list[k]["pid"]
    for s in sents_list[k]["sents"]:
        df.loc[i] = [pid, dec, s]
        i += 1

In [86]:
df.head(5)

,pid,dec,sent
0,2019_SJf_XhCqKm,Reject,The authors propose to use k-DPP to select a s...
1,2019_SJf_XhCqKm,Reject,"This paper covers the related work nicely, wit..."
2,2019_SJf_XhCqKm,Reject,The rest of the paper are also clearly written.
3,2019_SJf_XhCqKm,Reject,"However, I have some concerns about the propos..."
4,2019_SJf_XhCqKm,Reject,"- It is not clear how to define the kernel, th..."


In [87]:
df.to_excel("testset_16.xlsx")